In [1]:
import os
print("CWD:", os.getcwd())
print("FILES:", os.listdir())


CWD: C:\Users\chitl\Downloads\binvalidator_project
FILES: ['.ipynb_checkpoints', 'app.py', 'asin_label_encoder.pkl', 'bad_entries_log.json', 'best.pt', 'bin-images', 'clean_metadata_flat.csv', 'crops', 'data', 'debug_embeddings.py', 'eda.ipynb', 'flattened_metadata.csv', 'metadata']


In [2]:
import os
import json
from pathlib import Path
from PIL import Image
import pandas as pd
img_path = Path("bin-images")
meta_path = Path("metadata")


In [3]:
metadata_list = []
for file in meta_path.iterdir():
    if file.suffix == ".json":
        with open(file, "r") as f:
            data = json.load(f)
            data["metadata_file"] = file.name
            metadata_list.append(data)
df_meta = pd.DataFrame(metadata_list)
print("Loaded metadata files:", len(df_meta))
df_meta.head()

Loaded metadata files: 10000


,BIN_FCSKU_DATA,EXPECTED_QUANTITY,metadata_file,image_fname
0,"{'B000C33MI2': {'asin': 'B000C33MI2', 'height'...",12,00001.json,NaN
1,"{'0307981584': {'asin': '0307981584', 'height'...",17,00002.json,NaN
2,"{'0307981584': {'asin': '0307981584', 'height'...",16,00003.json,NaN
3,"{'B003E72M1G': {'asin': 'B003E72M1G', 'height'...",5,00004.json,NaN
4,"{'B003E72M1G': {'asin': 'B003E72M1G', 'height'...",4,00005.json,NaN


In [4]:
image_files = sorted([f.name for f in img_path.iterdir()
                      if f.suffix.lower() in [".jpg", ".jpeg", ".png"]])
print("Total images:", len(image_files))
image_files[:5]


Total images: 10000


['00001.jpg', '00002.jpg', '00003.jpg', '00004.jpg', '00005.jpg']

In [5]:
import json
print(json.dumps(df_meta["BIN_FCSKU_DATA"][0], indent=4))

{
    "B000C33MI2": {
        "asin": "B000C33MI2",
        "height": {
            "unit": "IN",
            "value": 2.79921259557
        },
        "length": {
            "unit": "IN",
            "value": 3.90157479917
        },
        "name": "FRAM XG7317 ULTRA Spin-On Oil Filter with Sure Grip",
        "normalizedName": "FRAM XG7317 ULTRA Spin-On Oil Filter with Sure Grip",
        "quantity": 4,
        "weight": {
            "unit": "pounds",
            "value": 0.3000050461296
        },
        "width": {
            "unit": "IN",
            "value": 2.90157480019
        }
    },
    "B0050Z27KG": {
        "asin": "B0050Z27KG",
        "height": {
            "unit": "IN",
            "value": 0.899999999082
        },
        "length": {
            "unit": "IN",
            "value": 11.299999988474
        },
        "name": "Suncatcher - Axicon Rainbow Window - Includes Bonus \"Rainbow on Board\" Sun Catcher",
        "normalizedName": "Suncatcher - Axicon Rainbo

In [6]:
df_meta.isnull().sum()

BIN_FCSKU_DATA          0
EXPECTED_QUANTITY       0
metadata_file           0
image_fname          9999
dtype: int64

In [7]:
df_meta["image_fname"] = df_meta["metadata_file"].str.replace(".json", ".jpg")

In [8]:
flattened_rows = []
for row in df_meta.itertuples():
    for asin, item in row.BIN_FCSKU_DATA.items():
        height = item.get("height") or {}
        length = item.get("length") or {}
        width  = item.get("width")  or {}
        weight = item.get("weight") or {}
        flattened_rows.append({
            "metadata_file": row.metadata_file,
            "asin": asin,
            "name": item.get("name"),
            "normalizedName": item.get("normalizedName"),
            "quantity": item.get("quantity"),

            "height_value": height.get("value"),
            "height_unit": height.get("unit"),

            "length_value": length.get("value"),
            "length_unit": length.get("unit"),

            "width_value": width.get("value"),
            "width_unit": width.get("unit"),

            "weight_value": weight.get("value"),
            "weight_unit": weight.get("unit"),
        })

df_items = pd.DataFrame(flattened_rows)
df_items.head()


,metadata_file,asin,name,normalizedName,quantity,height_value,height_unit,length_value,length_unit,width_value,width_unit,weight_value,weight_unit
0,00001.json,B000C33MI2,FRAM XG7317 ULTRA Spin-On Oil Filter with Sure...,FRAM XG7317 ULTRA Spin-On Oil Filter with Sure...,4,2.799213,IN,3.901575,IN,2.901575,IN,0.300005,pounds
1,00001.json,B0050Z27KG,Suncatcher - Axicon Rainbow Window - Includes ...,Suncatcher - Axicon Rainbow Window - Includes ...,2,0.900000,IN,11.300000,IN,7.700000,IN,0.500000,pounds
2,00001.json,B01BV89HNU,"Type C, iOrange-E 2 Pack 6.6 Ft Braided Cable ...","2 Pack Type C, iOrange-E 6.6 Ft Braided Cable ...",6,2.200000,IN,4.000000,IN,3.900000,IN,0.350000,pounds
3,00002.json,0307981584,Thomas Saves Easter! (Thomas & Friends) (Glitt...,Thomas Saves Easter! (Thomas & Friends) (Glitt...,13,0.400000,IN,7.900000,IN,6.900000,IN,0.400000,pounds
4,00002.json,B000052XHI,"First Response Early Result Pregnancy Test, 3 ...","First Response Early Result Pregnancy Test, 3 ...",2,1.600000,IN,7.800000,IN,3.300000,IN,0.022046,pounds


In [9]:
df_items.to_csv("flattened_metadata.csv", index=False)
print("CSV file saved as flattened_metadata.csv")


CSV file saved as flattened_metadata.csv


In [10]:
df = pd.read_csv("flattened_metadata.csv")
df.head()


,metadata_file,asin,name,normalizedName,quantity,height_value,height_unit,length_value,length_unit,width_value,width_unit,weight_value,weight_unit
0,00001.json,B000C33MI2,FRAM XG7317 ULTRA Spin-On Oil Filter with Sure...,FRAM XG7317 ULTRA Spin-On Oil Filter with Sure...,4,2.799213,IN,3.901575,IN,2.901575,IN,0.300005,pounds
1,00001.json,B0050Z27KG,Suncatcher - Axicon Rainbow Window - Includes ...,Suncatcher - Axicon Rainbow Window - Includes ...,2,0.900000,IN,11.300000,IN,7.700000,IN,0.500000,pounds
2,00001.json,B01BV89HNU,"Type C, iOrange-E 2 Pack 6.6 Ft Braided Cable ...","2 Pack Type C, iOrange-E 6.6 Ft Braided Cable ...",6,2.200000,IN,4.000000,IN,3.900000,IN,0.350000,pounds
3,00002.json,0307981584,Thomas Saves Easter! (Thomas & Friends) (Glitt...,Thomas Saves Easter! (Thomas & Friends) (Glitt...,13,0.400000,IN,7.900000,IN,6.900000,IN,0.400000,pounds
4,00002.json,B000052XHI,"First Response Early Result Pregnancy Test, 3 ...","First Response Early Result Pregnancy Test, 3 ...",2,1.600000,IN,7.800000,IN,3.300000,IN,0.022046,pounds


In [11]:
df.isnull().sum()


metadata_file       0
asin                0
name                6
normalizedName      6
quantity            0
height_value       44
height_unit        44
length_value       44
length_unit        44
width_value        44
width_unit         44
weight_value      609
weight_unit       609
dtype: int64

In [12]:
df.shape


(26421, 13)

In [13]:
df["asin"].nunique()


16472

In [14]:
df["name"].value_counts().head(20)


name
TaoTronics TT-AH002 30W Ultrasonic Humidifier with Cool Mist, Classic Dial Knob Control, 3.5L Large Capacity, Two 360 degree Rotatable Outlets                                                        19
19Watt 5/6-inch High CRI Wet Location Dimmable Retrofit LED Recessed Lighting Fixture, Energy Star UL-classified 120W Equivalent Ceiling Light, 2700K Soft White 1200lm Remodel Recessed Downlight    12
Fujifilm Instax Mini Instant Film (3 Twin Packs, 60 Total Pictures) Value Set                                                                                                                         11
My Little Pony POP and Starter Kits                                                                                                                                                                   11
Ever After High Epic Winter Ashlynn Ella Doll                                                                                                                                                  

In [15]:
df["height_unit"] = df["height_unit"].fillna("IN")
df["length_unit"] = df["length_unit"].fillna("IN")
df["width_unit"] = df["width_unit"].fillna("IN")
df["weight_unit"] = df["weight_unit"].fillna("pounds")


In [16]:
df["height_value"] = df.groupby("asin")["height_value"].transform(
    lambda x: x.fillna(x.median())
)
df["height_value"] = df["height_value"].fillna(df["height_value"].median())


In [17]:
df["length_value"] = df.groupby("asin")["length_value"].transform(
    lambda x: x.fillna(x.median())
)
df["length_value"] = df["length_value"].fillna(df["length_value"].median())


In [18]:
df["width_value"] = df.groupby("asin")["width_value"].transform(
    lambda x: x.fillna(x.median())
)
df["width_value"] = df["width_value"].fillna(df["width_value"].median())


In [19]:
df["weight_value"] = df.groupby("asin")["weight_value"].transform(
    lambda x: x.fillna(x.median())
)
df["weight_value"] = df["weight_value"].fillna(df["weight_value"].median())


In [20]:
df.isnull().sum()


metadata_file     0
asin              0
name              6
normalizedName    6
quantity          0
height_value      0
height_unit       0
length_value      0
length_unit       0
width_value       0
width_unit        0
weight_value      0
weight_unit       0
dtype: int64